# Introduction
This notebook shows how to connect to Alpaca and receive live price data from it. We run a strategy using this live data feed for 10 minutes and then show some results.  

In [ ]:
@file:Repository("https://roboquant.jfrog.io/artifactory/roboquant")
@file:DependsOn("org.roboquant:roboquant-extra:0.8-SNAPSHOT")

In [ ]:
Welcome()

By default the Alpaca feed classes are not imported, so we need to do that first.

In [ ]:
import org.roboquant.feeds.alpaca.*

# Setup
We now create, as usual, the strategy we want to test. There is nothing specific to Alapca that the strategy needs to be aware of. 

In [ ]:
val strategy = TAStrategy(12)

strategy.buy { 
    (ta.cdlMorningStar(it) || ta.cdlMorningDojiStar(it)) && ta.ema(it.close, 5) > ta.ema(it.close, 9)
}

strategy.sell { 
    ta.cdl2Crows(it) && ta.ema(it.close, 3) < ta.ema(it.close, 7)
}



We create an instance of Roboquant with the strategy we just defined and some metrics to later see how we have done. Again this is all independent of the data feed we are going to use

In [ ]:
val roboquant =  Roboquant(strategy, AccountSummary(), ProgressMetric())

# Feed
Now we create an instance of the AlpacaFeed and subscribe to one or more assets. We need to provide the API Key and Secret to authenticate against the Alpaca API. We can do this by:

* Setting the environment or JVM variables APCA_API_KEY_ID and APCA_API_SECRET_KEY
* Or pass them as parameters to the *AlpacaFeed* constructor. If you take this approach, make sure you don't accidentally share your notebook with anyone.

Also if you are going to test this yourself, make sure you use KEY/SECRET for papertrading.


In [ ]:
val feed = AlpacaFeed("your api key", "you secret")

Alpaca offers the prices for many stocks. For this notebook we're going to limit that a bit, we just get all the stocks who's symboal name starts with "AA" and then subscribe to them. Alpaca has a generous API rate limit, so it is possible for example to subscribe to all S&P 500 assets.

In [ ]:
val assets = feed.assets.filter { it.symbol.startsWith("AA") }
assets.summary()

In [ ]:
feed.subscribe(assets)

# Run
We have all the components we need to start the test. All that remains, is to start the run and evaluate the strategy against the feed. 

We'll run it for 10 minutes. If the time displayed looks wrong, don't forget that roboquant uses a timezone independent representation for all internal time processing (same as UTC).

In [ ]:
val timeFrame = TimeFrame.nextMinutes(10)
roboquant.run(feed, timeFrame)

# Disconnect
And we disconnect from the exchnage

In [ ]:
feed.disconnect()

In [ ]:
roboquant.broker.account.summary()

In [ ]:
roboquant.logger.summary()